In [21]:
import torch
from torch.autograd import Variable
import os
from metal.mmtl.cxr.analysis.notebooks.cxr_analysis import load_log_json, load_results_from_log
from metal.mmtl.cxr.cxr_tasks import create_tasks_and_payloads

In [103]:
#model_log = '/home/jdunnmon/Research/repos/metal/logs/2019_04_28/test_pneumo_slice_model_naive_10_22_22'
#model_log = '/home/jdunnmon/Research/repos/metal/logs/2019_04_28/test_pneumo_slice_model_hps_10_19_59'
model_log = '/home/jdunnmon/Research/repos/metal/logs/2019_04_28/test_pneumo_slice_model_22_19_24'
model = torch.load(os.path.join(model_log,'model.pkl'))

/home/jdunnmon/Research/repos/anaconda3/envs/metal/lib/python3.6/site-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'torchvision.models.densenet.DenseNet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [104]:
os.environ['CXRDATA']='/data/datasets/mmtl/cxr/'
os.environ['CXR8IMAGES']='/data/datasets/nih/images/images'
log_jsons = load_results_from_log(model_log)
task_names = ['CXR8-DRAIN_ALL']
tasks, payloads = create_tasks_and_payloads(task_names, **log_jsons['task_config'], batch_size=1)

Loading CXR8-DRAIN Dataset
Using resolution 224...
Using dataset seed: 1701
Using resolution 224...
Using dataset seed: 1701


/home/jdunnmon/Research/repos/anaconda3/envs/metal/lib/python3.6/site-packages/torchvision/transforms/transforms.py:208: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


Using resolution 224...
Using dataset seed: 1701
Added label_set with 78468/78468 labels for task CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:ind to payload CXR8-DRAIN_train.
Added label_set with 62608/78468 labels for task CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred to payload CXR8-DRAIN_train.
Added label_set with 78468/78468 labels for task CXR8-DRAIN_PNEUMOTHORAX_slice:BASE:ind to payload CXR8-DRAIN_train.
Added label_set with 78468/78468 labels for task CXR8-DRAIN_PNEUMOTHORAX_slice:BASE:pred to payload CXR8-DRAIN_train.
Added label_set with 11259/11259 labels for task CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:ind to payload CXR8-DRAIN_valid.
Added label_set with 208/11259 labels for task CXR8-DRAIN_PNEUMOTHORAX_slice:chest_drain_cnn_neg:pred to payload CXR8-DRAIN_valid.
Added label_set with 11259/11259 labels for task CXR8-DRAIN_PNEUMOTHORAX_slice:BASE:ind to payload CXR8-DRAIN_valid.
Added label_set with 11259/11259 labels for task CXR8-DRAIN_PNEUMOTHORA

In [105]:
dl = payloads[0].data_loader
batch = next(iter(dl))

In [106]:
from torchviz import make_dot
x,y = batch
out = model(x, task_names=set(payloads[0].labels_to_tasks.values()))

In [107]:
out_var = torch.sum(torch.stack([v['data'] for k,v in out.items()]))

In [108]:
#prms = {k:v for k,v in model.named_parameters() if 'input_module' not in k}

In [115]:
a = make_dot(out['CXR8-DRAIN_PNEUMOTHORAX_slice:BASE:pred']['data'], params = dict(model.named_parameters()))

In [116]:
a.render(filename='test-slice-viz-pneumo-BASE-sps',format='eps')

'test-slice-viz-pneumo-BASE-sps.eps'

In [77]:
a.render?

Signature:
a.render(
    filename=None,
    directory=None,
    view=False,
    cleanup=False,
    format=None,
    renderer=None,
    formatter=None,
)
Docstring:
Save the source to file and render with the Graphviz engine.

Args:
    filename: Filename for saving the source (defaults to ``name`` + ``'.gv'``)
    directory: (Sub)directory for source saving and rendering.
    view (bool): Open the rendered result with the default application.
    cleanup (bool): Delete the source file after rendering.
    format: The output format used for rendering (``'pdf'``, ``'png'``, etc.).
    renderer: The output renderer used for rendering (``'cairo'``, ``'gd'``, ...).
    formatter: The output formatter used for rendering (``'cairo'``, ``'gd'``, ...).
Returns:
    The (possibly relative) path of the rendered file.
Raises:
    ValueError: If ``format``, ``renderer``, or ``formatter`` are not known.
    graphviz.RequiredArgumentError: If ``formatter`` is given but ``renderer`` is None.
    graph